In [16]:
import keras
import numpy as np
import pandas as pd
import tensorflow as tf
import plotly.graph_objects as go
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.preprocessing.sequence import TimeseriesGenerator

In [17]:
filename = "Case_PRB_Europe_Only_true_Steven_Testing_Berat told me to_V0.17.csv"
df = pd.read_csv(filename)

look_back = 4
print(df.info())
df = df.groupby("location")
df = df.get_group("Albania")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10498 entries, 0 to 10497
Columns: 188 entries, Unnamed: 0 to day_int.1
dtypes: float64(15), int64(169), object(4)
memory usage: 15.1+ MB
None


In [18]:
df["date"] = pd.to_datetime(df["date"], infer_datetime_format=True)
df.set_axis(df["date"], inplace = True)

In [19]:
death_data = df["total_deaths"].values
death_data = death_data.reshape((-1, 1))

split_percent = 0.75
split = int(split_percent*len(death_data))

death_train = death_data[:split]
death_test = death_data[split:]

date_train = df["date"][:split]
date_test = df["date"][split:]

# print(death_data)
print(len(death_train))
print(len(death_test))

163
55


In [20]:
train_generator = TimeseriesGenerator(death_train, death_train, length=look_back, batch_size=20)
test_generator = TimeseriesGenerator(death_test, death_test, length=look_back, batch_size=1)

In [21]:
model = Sequential()
model.add(LSTM(64, activation='relu', input_shape=(look_back,1)))
model.add(Dropout(rate=0.2))
model.add(Dense(1))
model.compile(optimizer = "adam", loss="mse", metrics = ["accuracy"])

model.fit_generator(train_generator, epochs=30, verbose=1)


C:\Users\Gebruiker\AppData\Local\Temp\ipykernel_18972\1823632347.py:7: UserWarning:

`Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.



Epoch 1/30
8/8 [==============================] - 1s 3ms/step - loss: 7643.1187 - accuracy: 0.0000e+00
Epoch 2/30
8/8 [==============================] - 0s 3ms/step - loss: 6501.3057 - accuracy: 0.0000e+00
Epoch 3/30
8/8 [==============================] - 0s 3ms/step - loss: 5746.0493 - accuracy: 0.0000e+00
Epoch 4/30
8/8 [==============================] - 0s 3ms/step - loss: 4516.3911 - accuracy: 0.0000e+00
Epoch 5/30
8/8 [==============================] - 0s 3ms/step - loss: 2514.1558 - accuracy: 0.0063
Epoch 6/30
8/8 [==============================] - 0s 3ms/step - loss: 348.5084 - accuracy: 0.0000e+00
Epoch 7/30
8/8 [==============================] - 0s 3ms/step - loss: 228.9422 - accuracy: 0.0126
Epoch 8/30
8/8 [==============================] - 0s 3ms/step - loss: 175.5681 - accuracy: 0.0063
Epoch 9/30
8/8 [==============================] - 0s 3ms/step - loss: 316.6445 - accuracy: 0.0000e+00
Epoch 10/30
8/8 [==============================] - 0s 3ms/step - loss: 189.3973 - accurac

In [22]:
prediction = model.predict(test_generator)

death_train = death_train.reshape((-1))
death_test = death_test.reshape((-1))
prediction = prediction.reshape((-1))

print(date_train)

trace1 = go.Scatter(
    x = date_train,
    y = death_train,
    mode = "lines",
    name = "Data"
)
trace2 = go.Scatter(
    x = date_test,
    y = prediction,
    mode = "lines",
    name = "Prediction"
)
trace3 = go.Scatter(
    x = date_test,
    y = death_test,
    mode="lines",
    name = "Ground Truth"
)
layout = go.Layout(
    title = "Google Stock",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()

date
2020-09-03   2020-09-03
2020-10-03   2020-10-03
2020-11-03   2020-11-03
2020-12-03   2020-12-03
2020-03-13   2020-03-13
                ...    
2020-08-14   2020-08-14
2020-08-15   2020-08-15
2020-08-16   2020-08-16
2020-08-17   2020-08-17
2020-08-18   2020-08-18
Name: date, Length: 163, dtype: datetime64[ns]
